In [1]:
import pandas as pd
import numpy as np
import os
import csv
import time
from sentence_transformers import SentenceTransformer, util

DATASET_PATH = "../generated_data/quotes-2020-domains.json.bz2"
quotebank_2020_sample = pd.read_json(DATASET_PATH, lines=True, compression='bz2')
quotebank_2020_sample.head(10)
#max_corpus_size = 50000 # We limit our corpus to only the first 50k questions

In [2]:
quotebank_2020_sample.head(10)

NameError: name 'quotebank_2020_sample' is not defined

In [3]:
quotebank_sample = pd.read_json('../Valerian/Data_sample/quotes-2019-nytimes.json.bz2', lines=True, compression='bz2')
quotebank_sample.head(10)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2019-04-17-024782,"It is not a low-income immigration,",James Fisher,"[Q16213953, Q20707104, Q43143598, Q58886302, Q...",2019-04-17 13:31:18,1,"[[James Fisher, 0.7475], [None, 0.2525]]",[https://www.nytimes.com/2019/04/17/realestate...,E
1,2019-04-02-001128,a champion figure skater switching to roller s...,John Updike,[Q105756],2019-04-02 14:58:33,2,"[[John Updike, 0.5856], [None, 0.4144]]",[https://www.nytimes.com/2019/04/02/opinion/vl...,E
2,2019-05-09-055187,It makes it much more difficult for him to mak...,None,[],2019-05-09 18:11:29,1,"[[None, 0.6493], [President Bill Clinton, 0.27...",[http://mobile.nytimes.com/2019/05/09/world/as...,E
3,2019-10-31-056366,"It puts me in a predicament,",Xavier Becerra,[Q1855840],2019-10-31 16:45:15,3,"[[Xavier Becerra, 0.9065], [None, 0.0909], [St...",[http://www.nytimes.com/2019/10/31/technology/...,E
4,2019-01-04-001792,A Pile of Leaves.,None,[],2019-01-04 10:00:07,1,"[[None, 0.8737], [Jason Fulford, 0.1263]]",[https://www.nytimes.com/2019/01/04/books/revi...,E
5,2019-08-15-002017,A Senator we can call our own.,Tom Rath,[Q7817334],2019-08-15 22:36:33,1,"[[Tom Rath, 0.7598], [None, 0.1993], [Warren R...",[http://www.nytimes.com/2019/08/15/us/politics...,E
6,2019-02-12-002452,"a thorn in the side of cabinet,",None,[],2019-02-12 16:43:04,8,"[[None, 0.463], [Jody Wilson-Raybould, 0.4073]...",[https://castlegarsource.com/news/letter-apolo...,E
7,2019-01-28-048557,"It was just on a lark,",None,[],2019-01-28 18:32:47,2,"[[None, 0.5762], [Alexander the Great, 0.2153]...",[http://www.nytimes.com/2019/01/28/health/micr...,E
8,2019-07-22-032883,"It's a success, a relief and a technical feat,",Florence Parly,[Q3074013],2019-07-22 02:37:50,21,"[[Florence Parly, 0.9262], [None, 0.0738]]",[http://www.breitbart.com/news/french-submarin...,E
9,2019-12-31-002666,always appreciates and respects the work and a...,None,[],2019-12-31 22:09:50,3,"[[None, 0.8953], [President Trump, 0.1047]]",[http://www.ibtimes.com/climate-change-epa-sci...,E


In [5]:
quotebank_sample.shape

(207527, 9)

In [14]:

"""
This is a more complex example on performing clustering on large scale dataset.
This examples find in a large set of sentences local communities, i.e., groups of sentences that are highly
similar. You can freely configure the threshold what is considered as similar. A high threshold will
only find extremely similar sentences, a lower threshold will find more sentence that are less similar.
A second parameter is 'min_community_size': Only communities with at least a certain number of sentences will be returned.
The method for finding the communities is extremely fast, for clustering 50k sentences it requires only 5 seconds (plus embedding comuptation).
In this example, we download a large set of questions from Quora and then find similar questions in this set.
"""

# Model for computing sentence embeddings. We use one trained for similar questions detection
model = SentenceTransformer('all-MiniLM-L6-v2')

sentences = [ 'saving Muslim women from Muslim men',
'oppression of Muslim women',
'those poor Muslim women',
'Are Muslim Women Oppressed?',
'Demonising Muslim women, the small group of Muslim women who wear this, it is more oppressive',
'criminalise Muslim men in the guise of protecting Muslim women',
'Instead of protecting Muslim women, the Bill criminalises Muslim men',
'people (Muslim women) are suffering',
'Do Muslim Women Need Saving?',
'Muslim women are particularly vulnerable',
'traditional submissiveness of Muslim women',
'a patriarchal institution devoted to the oppression and enslavement of women and children.',
'the murder of innocent Muslims, the oppression of women, the persecution of Jews,',
'the oppression of women is a humanitarian problem, not just a women\'s problem',
'Oppression is deep, opposition is impossible, women are subject to deeply entrenched inequality.',
'We are a nation founded by men and women who fled religious oppression,',
'The original quote was intended to rouse women to stand up against oppression',
'It\'s a particularly awful and violent type of oppression that indigenous women experience,',
'ongoing and systemic oppression and criminalization of indigenous women and girls in Canada',
'Young gay women want no part of the male-female, oppression-submission roles', ]

quotes = quotebank_sample['quotation']
print("Encode the corpus. This might take a while")
sentences_embeddings = model.encode(sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)


print("Start clustering")
start_time = time.time()

#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least 25 elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(sentences_embeddings, min_community_size=25, threshold=0.75)

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", corpus_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", corpus_sentences[sentence_id])

Encode the corpus. This might take a while


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Start clustering


RuntimeError: selected index k out of range

In [15]:
model = SentenceTransformer('all-MiniLM-L6-v2')

sentences = [ 'saving Muslim women from Muslim men',
'oppression of Muslim women',
'those poor Muslim women',
'Are Muslim Women Oppressed?',
'Demonising Muslim women, the small group of Muslim women who wear this, it is more oppressive',
'criminalise Muslim men in the guise of protecting Muslim women',
'Instead of protecting Muslim women, the Bill criminalises Muslim men',
'people (Muslim women) are suffering',
'Do Muslim Women Need Saving?',
'Muslim women are particularly vulnerable',
'traditional submissiveness of Muslim women',
'a patriarchal institution devoted to the oppression and enslavement of women and children.',
'the murder of innocent Muslims, the oppression of women, the persecution of Jews,',
'the oppression of women is a humanitarian problem, not just a women\'s problem',
'Oppression is deep, opposition is impossible, women are subject to deeply entrenched inequality.',
'We are a nation founded by men and women who fled religious oppression,',
'The original quote was intended to rouse women to stand up against oppression',
'It\'s a particularly awful and violent type of oppression that indigenous women experience,',
'ongoing and systemic oppression and criminalization of indigenous women and girls in Canada',
'Young gay women want no part of the male-female, oppression-submission roles', ]

#Encode all sentences
embeddings = model.encode(sentences)

#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings, embeddings)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-5 most similar pairs:")
for score, i, j in all_sentence_combinations[0:5]:
    print("{} \t {} \t {:.4f}".format(sentences[i], sentences[j], cos_sim[i][j]))

Top-5 most similar pairs:
saving Muslim women from Muslim men 	 Do Muslim Women Need Saving? 	 0.8518
oppression of Muslim women 	 Are Muslim Women Oppressed? 	 0.8437
criminalise Muslim men in the guise of protecting Muslim women 	 Instead of protecting Muslim women, the Bill criminalises Muslim men 	 0.8253
oppression of Muslim women 	 Muslim women are particularly vulnerable 	 0.7819
people (Muslim women) are suffering 	 Muslim women are particularly vulnerable 	 0.7666
